![](https://digalyst.com/wp-content/uploads/2022/03/Incontent_image-1.png)
image from [link](https://digalyst.com/digital-marketing/mo-hinh-rfm-phan-tich-rfm-lam-marketing-phai-biet/)

## What is CRM and RFM? 

CRM is the management of a company's relationship with its customers.

RFM enables customers to be divided into groups based on their purchasing habits, and enables the development of strategies specific to each group.

##### RFM Metrics: 

* Recency: The time elapsed since the customer's last purchase.
* Frequency: The number of purchases made by the customer.
* Monetary: The total amount of money spent by the customer on purchases.

If we write these three metrics in the same language, we obtain the RFM score. 

We will create segments based on the scores obtained, using Recency and Frequency scores.



### Dataset Details (Retail Dataset)

InvoiceID : ID of the transaction. A transaction might hold multiple records for the same customer at the same date with multiple products (SKU). DocumentID might be useful for combining the transactions and detecting the items sold together.

Date : Date of transaction / sell. In the date time format.

ProductID : Item / Product code. The unique code for each item sold.

TotalSales : Sales price for the transaction. If you want to get unit_price , divide TotalSales column to Quantity column

Discount : Discount amount for the transaction.

CustomerID : Unique customer id for each customer. For the data set, customer can be a reseller or a branch of the company.

Quantity : Number of items sold in the transaction.

In [1]:
import datetime as dt
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df = pd.read_csv("/kaggle/input/retail-data-set/file_out2.csv")


In [2]:
# We are trying to understand the data.

def check_df(dataframe, head=7):
    print("################### Shape ####################")
    print(dataframe.shape)
    print("#################### Info #####################")
    print(dataframe.info())
    print("################### Nunique ###################")
    print(dataframe.nunique())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("################## Quantiles #################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    print("#################### Head ####################")
    print(dataframe.head(head))

check_df(df)

################### Shape ####################
(29103, 8)
#################### Info #####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29103 entries, 0 to 29102
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  29103 non-null  int64  
 1   InvoiceID   29103 non-null  int64  
 2   Date        29103 non-null  object 
 3   ProductID   29103 non-null  int64  
 4   TotalSales  29103 non-null  float64
 5   Discount    29103 non-null  float64
 6   CustomerID  29103 non-null  int64  
 7   Quantity    29103 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 1.8+ MB
None
################### Nunique ###################
Unnamed: 0    29103
InvoiceID     14079
Date           1268
ProductID      1940
TotalSales     7013
Discount       6797
CustomerID      507
Quantity         59
dtype: int64
##################### NA #####################
Unnamed: 0    0
InvoiceID     0
Date     

In [3]:
# Data Preparation

# We are finding the total price of the product.
df["TotalPrice"] = df["TotalSales"] + df["Discount"] 

# We are finding the unit price of the product.
df["Unit_Price"] = df["TotalPrice"] / df["Quantity"]

# Since the value of these variables cannot be 0,
# we are removing those with a value of 0 from the data.
df = df[(df['Quantity'] > 0)]
df = df[(df["TotalPrice"] > 0)]
df = df[(df['Discount'] > 0)]       

# We are deleting the variable that does not carry any information.
df.drop("Unnamed: 0", inplace=True, axis=1)

df.head()

,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity,TotalPrice,Unit_Price
0,328,2019-12-27,1684,796.610,143.390,185,4,940.000,235.000
1,329,2019-12-27,524,355.932,64.068,185,2,420.000,210.000
2,330,2019-12-27,192,901.695,162.305,230,4,1064.000,266.000
3,330,2019-12-27,218,182.754,32.896,230,1,215.650,215.650
4,330,2019-12-27,247,780.102,140.418,230,4,920.520,230.130


In [4]:
# Calculating RFM Metrics

# We are converting the object type date to the datetime structure.
df["Date"] = pd.to_datetime(df["Date"])    

df["Date"].max() 

# We are entering the date on which we prepared the data to calculate the Recency value
today_date = pd.to_datetime("2023-04-01")


rfm = df.groupby('CustomerID').agg({'Date': lambda Date: (today_date - Date.max()).days,    # recency degeri
                                     'InvoiceID': lambda Invoice: Invoice.nunique(),        # frequency degeri
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})    # monetary degeri


# We are changing the names of the variables we created.
rfm.columns = ['recency', 'frequency', 'monetary']

rfm.head()

,recency,frequency,monetary
CustomerID,,,
0,14,31,311001.360
1,742,13,24162.540
2,59,22,61317.180
3,632,1,355.000
4,99,35,71953.710


In [5]:
# Calculating RFM Scores

# Since the smallest value in Recency is the most valuable, we start scoring from 5
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])   

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])


# We are creating the RFM score.
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str)) 

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
CustomerID,,,,,,,
0,14,31,311001.360,5,4,5,54
1,742,13,24162.540,2,4,3,24
2,59,22,61317.180,4,4,4,44
3,632,1,355.000,2,1,1,21
4,99,35,71953.710,4,4,4,44


In [6]:
# Creating & Analysing RFM Segments

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}


# We are creating the correspondences of RFM scores in segments
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)     


# We are examining the RFM values corresponding to the segments
rfm.groupby("segment").agg({"recency":["mean","count"],
                            "frequency":["mean","count"],
                            "monetary":["mean","count"]})



recency       frequency         monetary      
                        mean count      mean count       mean count
segment                                                            
about_to_sleep       299.318    44     1.023    44   5337.898    44
at_Risk              993.855    69    11.159    69  18602.861    69
cant_loose           718.667     3   191.333     3 302146.723     3
champions             13.202    84   154.690    84 752947.106    84
hibernating         1090.805   128     1.016   128   2174.519   128
loyal_customers      137.483    87    61.989    87 215611.394    87
need_attention       255.519    27     3.926    27  13545.609    27
new_customers         15.500     2     1.000     2  12581.540     2
potential_loyalists   54.957    46     3.435    46  20871.481    46
promising             66.182    11     1.000    11   5826.251    11

In [7]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,
0,14,31,311001.360,5,4,5,54,champions
1,742,13,24162.540,2,4,3,24,at_Risk
2,59,22,61317.180,4,4,4,44,loyal_customers
3,632,1,355.000,2,1,1,21,hibernating
4,99,35,71953.710,4,4,4,44,loyal_customers


   ![](https://miro.medium.com/v2/resize:fit:786/format:webp/1*oUKMc3gk53hccFX4YNeu1g.png)

In [9]:
# We are creating a new dataframe.
new_df = pd.DataFrame()


# We can specify the desired segment in the new dataframe and access the IDs of the customers belonging to that segment
new_df["loyal_customers_id"] = rfm[rfm["segment"] == "loyal_customers"].index


new_df.head()

,loyal_customers_id
0,2
1,4
2,11
3,16
4,17
